In [1]:
import sys
sys.path.append('../..')

import csv
import codecs
import copy
import re
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk import word_tokenize
import gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.externals import joblib

from utils import tokenizer, load_six_emotions, load_3_emotions

%matplotlib inline

np.warnings.filterwarnings('ignore')
np.random.seed(12345)

def highlight_max(data, color='green'):
    attr = f'background-color: {color}; color: white; font-weight: bold;'
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

### Matriz de resultados

In [2]:
classf = {
    'MultinomialNB': 0,
    'ComplementNB': 0,
    'LogisticRegression': 0,
    'RandomForestClassifier': 0,
    'KNeighborsClassifier': 0,
    'MLPClassifier': 0,
    'LinearSVC': 0,
    'SVC': 0
}

matriz_resultados = {
    'tfidf': copy.deepcopy(classf),
    'tfidf+lsa': copy.deepcopy(classf),
    'tfidf+lda': copy.deepcopy(classf),
    'count': copy.deepcopy(classf),
    'count+lsa': copy.deepcopy(classf),
    'count+lda': copy.deepcopy(classf),
    'tfidf+count+w2c': copy.deepcopy(classf),
}

### Dataset

Fornecido pela equipe da PUC-PR através do site [EMOÇÕES.BR](http://www.ppgia.pucpr.br/~paraiso/mineracaodeemocoes/index.php), contém cerca de 1000 frases, categorizadas usando as 6 emoções de Ekman (alegria, surpresa, tristeza, medo, raiva, desgosto).
Alem disso, foi acrescido em torno de 700 novas frases rotuladas, totalizando 1720 frases.

In [3]:
def carregar(filename):
    frases = []
    with open(filename, 'r') as h:
        reader = csv.reader(h, delimiter='|')
        for row in reader:
            frase = tokenizer(row[0]).strip()
            sentimento = row[1].upper()
            if len(frase) > 5:
                frases.append((sentimento, frase))
    return frases

frases = carregar('/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/frases/frases_todas.txt')

shuffle(frases)
print(frases[:5])

[('SURPRESA', 'visã torn clar soment pod olhar própri coraçã olhar sonh olhar dentr despert'), ('MEDO', 'fim corredor nad além mim escuridã'), ('TRISTEZA', 'perdo tant doer tant final doer machuc'), ('SURPRESA', 'sab escut pod ter certez silênci produz muit vez efeit ciênc'), ('ALEGRIA', 'comprend pod só nã pod proteg direit faz quer')]


In [4]:
afrases = []
asentimentos =[]
for sentimento, frase in frases:
    afrases.append(frase)
    asentimentos.append(sentimento)

### Funções de Apoio

In [5]:
def run_ml_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    accuracy = np.round(model.score(X_test, y_test) * 100, 2)
    print(f'Modelo   : {model.__class__.__name__}')
    print(f'Acurácia : {accuracy}%')
    print('-' * 20)
    return accuracy

def split_data(X, y):
    test_size = .33
    random_state = 0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test
    }

### Classificadores

In [6]:
classifiers = (
    MultinomialNB(),
    ComplementNB(),
    LogisticRegression(multi_class='auto', solver='lbfgs'),
    RandomForestClassifier(n_estimators=50, min_samples_split=8, random_state=0),
    KNeighborsClassifier(n_neighbors=8, algorithm='auto'),
    MLPClassifier(hidden_layer_sizes=(100, 25), max_iter=500, random_state=0),
    LinearSVC(max_iter=500),
    SVC(gamma='auto', max_iter=500),
)

### TF-IDF

In [7]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

In [8]:
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf, asentimentos))
        matriz_resultados['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

Modelo   : MultinomialNB
Acurácia : 46.22%
--------------------
Modelo   : ComplementNB
Acurácia : 53.66%
--------------------
Modelo   : LogisticRegression
Acurácia : 51.8%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 51.18%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 48.82%
--------------------
Modelo   : MLPClassifier
Acurácia : 53.78%
--------------------
Modelo   : LinearSVC
Acurácia : 56.51%
--------------------
Modelo   : SVC
Acurácia : 26.39%
--------------------


### LSA (usando TF-IDF)

In [9]:
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, asentimentos))
        matriz_resultados['tfidf+lsa'][classifier.__class__.__name__] = acc
    except Exception as e:
        print(e)

Modelo   : MultinomialNB
Acurácia : 26.39%
--------------------
Modelo   : ComplementNB
Acurácia : 45.35%
--------------------
Modelo   : LogisticRegression
Acurácia : 54.77%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 53.66%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 47.58%
--------------------
Modelo   : MLPClassifier
Acurácia : 56.01%
--------------------
Modelo   : LinearSVC
Acurácia : 54.77%
--------------------
Modelo   : SVC
Acurácia : 26.39%
--------------------


### LDA (usando TF-IDF)

In [10]:
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_tfidf)

for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, asentimentos))
        matriz_resultados['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass

Modelo   : MultinomialNB
Acurácia : 27.14%
--------------------
Modelo   : ComplementNB
Acurácia : 21.31%
--------------------
Modelo   : LogisticRegression
Acurácia : 26.52%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 33.83%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 26.39%
--------------------
Modelo   : MLPClassifier
Acurácia : 28.13%
--------------------
Modelo   : LinearSVC
Acurácia : 25.77%
--------------------
Modelo   : SVC
Acurácia : 26.39%
--------------------


### Count

In [11]:
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count = vec_count.fit_transform(afrases)

In [12]:
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count, asentimentos))
        matriz_resultados['count'][classifier.__class__.__name__] = acc
    except:
        pass

Modelo   : MultinomialNB
Acurácia : 53.9%
--------------------
Modelo   : ComplementNB
Acurácia : 46.96%
--------------------
Modelo   : LogisticRegression
Acurácia : 55.64%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 52.29%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 26.64%
--------------------
Modelo   : MLPClassifier
Acurácia : 53.16%
--------------------
Modelo   : LinearSVC
Acurácia : 53.41%
--------------------
Modelo   : SVC
Acurácia : 26.39%
--------------------


### LSA (usando Count)

In [13]:
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(svd, normalizer)
X_svd = lda.fit_transform(X_count)

for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, asentimentos))
        matriz_resultados['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass

Modelo   : MultinomialNB
Acurácia : 26.39%
--------------------
Modelo   : ComplementNB
Acurácia : 39.16%
--------------------
Modelo   : LogisticRegression
Acurácia : 50.56%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 48.08%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 41.26%
--------------------
Modelo   : MLPClassifier
Acurácia : 47.34%
--------------------
Modelo   : LinearSVC
Acurácia : 50.93%
--------------------
Modelo   : SVC
Acurácia : 26.39%
--------------------


### LDA (usando Count)

In [14]:
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_count)

for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, asentimentos))
        matriz_resultados['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass

Modelo   : MultinomialNB
Acurácia : 28.38%
--------------------
Modelo   : ComplementNB
Acurácia : 19.95%
--------------------
Modelo   : LogisticRegression
Acurácia : 27.63%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 29.99%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 29.74%
--------------------
Modelo   : MLPClassifier
Acurácia : 29.99%
--------------------
Modelo   : LinearSVC
Acurácia : 26.89%
--------------------
Modelo   : SVC
Acurácia : 26.39%
--------------------


### Count + TF-IDF + Word2Vec

In [15]:
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(afrases)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(afrases)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in afrases:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(29456811, 31096000)

In [16]:
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

In [17]:
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, asentimentos))
        matriz_resultados['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass

Modelo   : LogisticRegression
Acurácia : 54.28%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 52.54%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 46.72%
--------------------
Modelo   : MLPClassifier
Acurácia : 49.07%
--------------------
Modelo   : LinearSVC
Acurácia : 55.14%
--------------------
Modelo   : SVC
Acurácia : 26.39%
--------------------


### Resultado dos Classificadores

In [18]:
df = pd.DataFrame.from_dict(matriz_resultados)
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,53.66%,45.35%,21.31%,46.96%,39.16%,19.95%,0.00%
KNeighborsClassifier,48.82%,47.58%,26.39%,26.64%,41.26%,29.74%,46.72%
LinearSVC,56.51%,54.77%,25.77%,53.41%,50.93%,26.89%,55.14%
LogisticRegression,51.80%,54.77%,26.52%,55.64%,50.56%,27.63%,54.28%
MLPClassifier,53.78%,56.01%,28.13%,53.16%,47.34%,29.99%,49.07%
MultinomialNB,46.22%,26.39%,27.14%,53.90%,26.39%,28.38%,0.00%
RandomForestClassifier,51.18%,53.66%,33.83%,52.29%,48.08%,29.99%,52.54%
SVC,26.39%,26.39%,26.39%,26.39%,26.39%,26.39%,26.39%


### Modelo escolhido e salvo

In [19]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

svm = LinearSVC(max_iter=1200)
model = CalibratedClassifierCV(svm) 
model.fit(X_svd, asentimentos)

accuracy = np.round(model.score(X_svd, asentimentos) * 100, 2)
print(f'Modelo   : {model.__class__.__name__}')
print(f'Acurácia : {accuracy}%')

filename = 'tfidf_emotions.sav'
joblib.dump(vec_tfidf, filename)

filename = 'lsa_emotions.sav'
joblib.dump(lsa, filename)

filename = 'model_emotions.sav'
joblib.dump(model, filename)

Modelo   : CalibratedClassifierCV
Acurácia : 57.61%


['model_emotions.sav']

In [20]:
model.classes_

array(['ALEGRIA', 'DESGOSTO', 'MEDO', 'NEUTRO', 'RAIVA', 'SURPRESA',
       'TRISTEZA'], dtype='<U8')

In [21]:
y = model.predict_proba(X_svd)
list(np.round(y[0] * 100, 2))

[22.49, 7.79, 11.42, 1.23, 8.77, 10.27, 38.04]